Part 4: Change Data Capture (CDC)

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy
import logging
from datetime import datetime

# -------------------------------
# Logging setup
# -------------------------------
logging.basicConfig(
    filename="etl_cdc.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# -------------------------------
# DB connection
# -------------------------------
engine = create_engine("sqlite:///sales_dw.db", echo=False)

# -------------------------------
# Ensure history table exists
# -------------------------------
with engine.begin() as conn:
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS sales_history (
        ChangeID INTEGER PRIMARY KEY AUTOINCREMENT,
        ChangeType TEXT,
        Date DATE,
        ProductID INTEGER,
        ProductName TEXT,
        QuantitySold INTEGER,
        Price REAL,
        Category TEXT,
        CustomerID INTEGER,
        TotalSales REAL,
        ChangeTimestamp DATETIME
    )
    """))

# -------------------------------
# Load updates file
# -------------------------------
updates = pd.read_csv("../data/sales_data_updates.csv")

# Compute TotalSales for applicable rows
updates.loc[updates["ChangeType"] != "DELETE", "TotalSales"] = (
    updates["QuantitySold"] * updates["Price"]
)

# -------------------------------
# Process CDC operations
# -------------------------------
with engine.begin() as conn:
    for _, row in updates.iterrows():
        change_type = row["ChangeType"].upper()
        key = {
            "Date": row["Date"],
            "ProductID": row["ProductID"],
            "CustomerID": row["CustomerID"],
        }

        # --- INSERT ---
        if change_type == "INSERT":
            conn.execute(text("""
                INSERT INTO fact_sales (DateID, ProductID, ProductName, QuantitySold, Price, Category, CustomerID, TotalSales)
                VALUES (:Date, :ProductID, :ProductName, :QuantitySold, :Price, :Category, :CustomerID, :TotalSales)
            """), row.to_dict())

        # --- UPDATE ---
        elif change_type == "UPDATE":
            # Save old record to history
            old = conn.execute(text("""
                SELECT * FROM fact_sales
                WHERE Date=:Date AND ProductID=:ProductID AND CustomerID=:CustomerID
            """), key).fetchone()

            if old:
                conn.execute(text("""
                    INSERT INTO sales_history (ChangeType, Date, ProductID, ProductName, QuantitySold, Price, Category, CustomerID, TotalSales, ChangeTimestamp)
                    VALUES ('UPDATE', :Date, :ProductID, :ProductName, :QuantitySold, :Price, :Category, :CustomerID, :TotalSales, :ts)
                """), {**dict(old), "ts": datetime.now()})

                # Apply update
                conn.execute(text("""
                    UPDATE fact_sales
                    SET ProductName=:ProductName,
                        QuantitySold=:QuantitySold,
                        Price=:Price,
                        Category=:Category,
                        TotalSales=:TotalSales
                    WHERE Date=:Date AND ProductID=:ProductID AND CustomerID=:CustomerID
                """), row.to_dict())

        # --- DELETE ---
        elif change_type == "DELETE":
            # Save old record to history
            old = conn.execute(text("""
                SELECT * FROM fact_sales
                WHERE Date=:Date AND ProductID=:ProductID AND CustomerID=:CustomerID
            """), key).fetchone()

            if old:
                conn.execute(text("""
                    INSERT INTO sales_history (ChangeType, Date, ProductID, ProductName, QuantitySold, Price, Category, CustomerID, TotalSales, ChangeTimestamp)
                    VALUES ('DELETE', :Date, :ProductID, :ProductName, :QuantitySold, :Price, :Category, :CustomerID, :TotalSales, :ts)
                """), {**dict(old), "ts": datetime.now()})

                # Delete from fact_sales
                conn.execute(text("""
                    DELETE FROM fact_sales
                    WHERE Date=:Date AND ProductID=:ProductID AND CustomerID=:CustomerID
                """), key)

        logging.info(f"Processed {change_type} for ProductID={row['ProductID']} on {row['Date']}")

print("CDC ETL complete.")


CDC ETL complete.


In [ ]:
# 4. Find the total sales made by each customer and list the top 3 customers based on TotalSales.
# NOTE: Not possible to update delete rows because recores are from January only. Initially added records are from February.

query = """
SELECT ProductID,	ProductName,	QuantitySold,	Price,	Category,	CustomerID,	TotalSales,	DateID
FROM fact_sales f
;
"""

result4_df = pd.read_sql_query(query, con=engine)
display(result4_df)

,ProductID,ProductName,QuantitySold,Price,Category,CustomerID,TotalSales,DateID
0,9,Gadget I,10,10.0,Gadgets,102,100.0,2023-02-01
1,10,Gizmo J,3,11.0,Gizmos,103,33.0,2023-02-02
2,1,Widget A,6,2.5,Gadgets,104,15.0,2023-02-03
3,2,Gadget B,9,3.0,Gadgets,105,27.0,2023-02-04
4,3,Widget C,8,4.0,Widgets,101,32.0,2023-02-05
5,4,Gizmo D,6,5.0,Gizmos,102,30.0,2023-02-06
6,5,Widget E,7,6.0,Widgets,103,42.0,2023-02-07
7,6,Gadget F,5,7.0,Gadgets,104,35.0,2023-02-08
8,7,Gizmo G,2,8.0,Gizmos,105,16.0,2023-02-09
9,8,Widget H,11,9.0,Widgets,101,99.0,2023-02-10
